In [ ]:
from keras.layers import Activation
from keras.layers import Lambda
from keras.layers import Conv2D
from keras.layers import Add, UpSampling2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import ZeroPadding2D
from keras.layers import Input
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
from keras.models import model_from_yaml
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
import tensorflow as tf
from PIL import Image
import numpy as np
import copy
import os
import cv2
import keras
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras import backend as K
%matplotlib inline

### Identify horizontally flipped images

In [ ]:
def check(path):
    head, tail=os.path.split(path)
    if tail.startswith('h'):
        return True
    else:
        return False

### Load image as numpy array

In [ ]:
def load_image(path):
    img=mpimg.imread(path)
    gate=check(path)
    if gate==True:
        img=img[120:600,0:800] # Appropriate crop for horizontal flipped image
    else:
        img=img[96:480,0:800]
    h, w, d = img.shape
    img=cv2.resize(img, dsize=((w//32)*32,(h//32)*32),interpolation=cv2.INTER_NEAREST) # Resizing the images in the factor of 32. Floor division of the original size and multiplying with 32
    img = np.array(img, dtype=np.float32)
    x = np.expand_dims(img, axis=0)
   
    return x

### Load label as numpy array compatible for training (three upcoming cells)

In [ ]:
# 1/4 size Label image preparation
def load_label(path):
    img=mpimg.imread(path)
    gate=check(path)
    if gate==True:
        img=img[120:600,0:800]
    else:
        img=img[96:480,0:800]
    h, w, d=img.shape
    img_new=cv2.resize(img, dsize=((w//32)*8,(h//32)*8),interpolation=cv2.INTER_NEAREST)
    img_decoded=img_new[:,:,0]*255
    img_decoded=img_decoded.astype(int)
    img_coded=np.zeros_like(img_new[:,:,0])
    for i in range((img_new.shape[0])):
        for j in range((img_new.shape[1])):
            if img_decoded[i][j]==10.0:            
                img_coded[i][j]=2

            elif img_decoded[i][j]==7.0:
                img_coded[i][j]=1
                
            else:
                img_coded[i][j]=0
            
    img_coded=np.array(img_coded,dtype=np.uint8)
    y=np.zeros((1,img_new.shape[0],img_new.shape[1],3),dtype=np.float32)
    for i in range(img_new.shape[0]):
        for j in range(img_new.shape[1]):
            y[0,i,j,img_coded[i][j]]=1
            
    return y

In [ ]:
# 1/8 size Label image preparation
def load_label1(path):
    img=mpimg.imread(path)
    gate=check(path)
    if gate==True:
        img=img[120:600,0:800]
    else:
        img=img[96:480,0:800]
    h, w, d=img.shape
    img_new=cv2.resize(img, dsize=((w//32)*4,(h//32)*4),interpolation=cv2.INTER_NEAREST)
    img_decoded=img_new[:,:,0]*255
    img_decoded=img_decoded.astype(int)
    img_coded=np.zeros_like(img_new[:,:,0])
    for i in range((img_new.shape[0])):
        for j in range((img_new.shape[1])):
            if img_decoded[i][j]==10.0:            
                img_coded[i][j]=2

            elif img_decoded[i][j]==7.0:
                img_coded[i][j]=1
                
            else:
                img_coded[i][j]=0
            
    img_coded=np.array(img_coded,dtype=np.uint8)
    y=np.zeros((1,img_new.shape[0],img_new.shape[1],3),dtype=np.float32)
    for i in range(img_new.shape[0]):
        for j in range(img_new.shape[1]):
            y[0,i,j,img_coded[i][j]]=1                
    return y

In [ ]:
# 1/16 size Label image preparation
def load_label2(path):
    img=mpimg.imread(path)
    gate=check(path)
    if gate==True:
        img=img[120:600,0:800]
    else:
        img=img[96:480,0:800]
    h, w, d=img.shape
    img_new=cv2.resize(img, dsize=((w//32)*2,(h//32)*2),interpolation=cv2.INTER_NEAREST)
    img_decoded=img_new[:,:,0]*255
    img_decoded=img_decoded.astype(int)
    img_coded=np.zeros_like(img_new[:,:,0])
    for i in range((img_new.shape[0])):
        for j in range((img_new.shape[1])):
            if img_decoded[i][j]==10.0:            
                img_coded[i][j]=2

            elif img_decoded[i][j]==7.0:
                img_coded[i][j]=1
                
            else:
                img_coded[i][j]=0
            
    img_coded=np.array(img_coded,dtype=np.uint8)
    y=np.zeros((1,img_new.shape[0],img_new.shape[1],3),dtype=np.float32)
    for i in range(img_new.shape[0]):
        for j in range(img_new.shape[1]):
            y[0,i,j,img_coded[i][j]]=1
    return y

### Generate arrays from images and labels in the path 

In [ ]:
def generate_arrays_from_file(path, image_dir, label_dir):
    while 1:
        f = open(path)
        for line in f:
            filename = line.rstrip('\n')
            path_image = os.path.join(image_dir, filename)
            path_label = os.path.join(label_dir, filename)
            x = load_image(path_image)
            w = load_label(path_label)
            y = load_label1(path_label)
            z = load_label2(path_label)
            yield (x, [w, y, z])
        f.close()

### Resize tensors for the network (three upcoming cells)

In [ ]:
# Reduce the size to half
def Resizer1(img):
    from keras.backend import tf as ktf
    return_image=ktf.image.resize_bilinear(img, size=(int(img.shape[1])//2, int(img.shape[2])//2))   
    return return_image

In [ ]:
#Reduce the size based on argument
def Resizer2(img,value):
    from keras.backend import tf as ktf
    return_image=ktf.image.resize_bilinear(img,size=value)    
    return return_image

In [ ]:
#Increase the size (doubling)
def Resizer3(img):
    from keras.backend import tf as ktf
    return_image=ktf.image.resize_bilinear(img,size=(int(img.shape[1])*2,int(img.shape[2])*2))    
    return return_image

### Neural Network (ICNet - Image Cascade Network) 

In [ ]:
def network(width,height,train=False):  
    import tensorflow as tf
    inp = Input(shape=(height,width, 3))
    x = Lambda(lambda x: x)(inp)

    # (1/2)
    y = Lambda(Resizer1, name='Resize1')(x)
    y = Conv2D(32, 3, strides=2, padding='same', activation='relu', name='Conv1/2_1')(y)
    y = BatchNormalization(name='BN_1')(y)
    y = Conv2D(32, 3, padding='same', activation='relu', name='conv1/2_2')(y)
    y = BatchNormalization(name='BN_2')(y)
    y = Conv2D(64, 3, padding='same', activation='relu', name='conv1/2_3')(y)
    y = BatchNormalization(name='BN_3')(y)
    y_ = MaxPooling2D(pool_size=3, strides=2, name='Pool_1/2_1')(y)
    
    y = Conv2D(64, 1, activation='relu', name='Conv1/2_4')(y_)
    y = BatchNormalization(name='BN_4')(y)
    y = ZeroPadding2D(name='Padding1')(y)
    y = Conv2D(32, 3, activation='relu', name='conv1/2_5')(y)
    y = BatchNormalization(name='BN_5')(y)
    y = Conv2D(64, 1, name='conv1/2_6')(y)
    y = BatchNormalization(name='BN_6')(y)
    y = Add(name='Add1')([y,y_])
    y_ = Activation('relu', name='RELU_1')(y)

    y = Conv2D(256, 1, strides=2, name='Conv1/2_7')(y_)
    y = BatchNormalization(name='BN_7')(y)
    y_ = Conv2D(64, 1, strides=2, activation='relu', name='Conv1/2_8')(y_)
    y_ = BatchNormalization(name='BN_8')(y_) 
    y_ = ZeroPadding2D(name='Padding2')(y_)
    y_ = Conv2D(64, 3, activation='relu', name='Conv1/2_9')(y_)
    y_ = BatchNormalization(name='BN_9')(y_)
    y_ = Conv2D(256, 1, name='Conv1/2_10')(y_)
    y_ = BatchNormalization(name='BN_10')(y_)
    y = Add(name='Add2')([y,y_])
    z = Activation('relu', name='RELU_2')(y)

    # (1/4)
    y_ = Lambda(Resizer1, name='Resizer2')(z)
    y = Conv2D(64, 1, activation='relu', name='Conv1/4_1')(y_)
    y = BatchNormalization(name='BN_11')(y)
    y = ZeroPadding2D(name='Padding3')(y)
    y = Conv2D(64, 3, activation='relu', name='Conv1/4_2')(y)
    y = BatchNormalization(name='BN_12')(y)
    y = Conv2D(256, 1, name='Conv1/4_3')(y)
    y = BatchNormalization(name='BN_13')(y)
    y = Add(name='Add3')([y,y_])
    y_ = Activation('relu', name='RELU_3')(y)

    y = Conv2D(256, 1, activation='relu', name='Conv1/4_4')(y_)
    y = BatchNormalization(name='BN_14')(y)
    y = ZeroPadding2D(name='Padding4')(y)
    y = Conv2D(64, 3, activation='relu', name='Conv1/4_5')(y)
    y = BatchNormalization(name='BN_15')(y)
    y = Conv2D(256, 1, name='Conv1/4_6')(y)
    y = BatchNormalization(name='BN_16')(y)
    y = Add(name='Add4')([y,y_])
    y_ = Activation('relu', name='RELU_4')(y)

    y = Conv2D(512, 1, name='Conv1/4_7')(y_)
    y = BatchNormalization(name='BN_17')(y)
    y_ = Conv2D(128, 1, activation='relu', name='Conv1/4_8')(y_)
    y_ = BatchNormalization(name='BN_18')(y_)
    y_ = ZeroPadding2D(padding=2, name='Padding5')(y_)
    y_ = Conv2D(128, 3, dilation_rate=2, activation='relu', name='Conv1/4_9')(y_)
    y_ = BatchNormalization(name='BN_19')(y_)
    y_ = Conv2D(512, 1, name='Conv1/4_10')(y_)
    y_ = BatchNormalization(name='BN_20')(y_)
    y = Add(name='Add5')([y,y_])
    y_ = Activation('relu', name='RELU_5')(y)

    y = Conv2D(128, 1, activation='relu', name='Conv1/4_11')(y_)
    y = BatchNormalization(name='BN_21')(y)
    y = ZeroPadding2D(padding=2, name='Padding6')(y)
    y = Conv2D(128, 3, dilation_rate=2, activation='relu', name='Conv1/4_12')(y)
    y = BatchNormalization(name='BN_22')(y)
    y = Conv2D(512, 1, name='Conv1/4_13')(y)
    y = BatchNormalization(name='BN_23')(y)
    y = Add(name='Add6')([y,y_])
    y_ = Activation('relu', name='RELU_6')(y)

    y = Conv2D(128, 1, activation='relu', name='Conv1/4_14')(y_)
    y = BatchNormalization(name='BN_24')(y)
    y = ZeroPadding2D(padding=2, name='Padding7')(y)
    y = Conv2D(128, 3, dilation_rate=2, activation='relu', name='Conv1/4_15')(y)
    y = BatchNormalization(name='BN_25')(y)
    y = Conv2D(512, 1, name='Conv1/4_16')(y)
    y = BatchNormalization(name='BN_26')(y)
    y = Add(name='Add7')([y,y_])
    y_ = Activation('relu', name='RELU_7')(y)

    y = Conv2D(128, 1, activation='relu', name='Conv1/4_17')(y_)
    y = BatchNormalization(name='BN_27')(y)
    y = ZeroPadding2D(padding=2, name='Padding8')(y)
    y = Conv2D(128, 3, dilation_rate=2, activation='relu', name='Conv1/4_18')(y)
    y = BatchNormalization(name='BN_28')(y)
    y = Conv2D(512, 1, name='Conv1/4_19')(y)
    y = BatchNormalization(name='BN_29')(y)
    y = Add(name='Add8')([y,y_])
    y = Activation('relu', name='RELU_8')(y)

    y_ = Conv2D(1024, 1, name='Conv1/4_20')(y)
    y_ = BatchNormalization(name='BN_30')(y_)
    y = Conv2D(256, 1, activation='relu', name='Conv1/4_21')(y)
    y = BatchNormalization(name='BN_31')(y)
    y = ZeroPadding2D(padding=4, name='Padding9')(y)
    y = Conv2D(256, 3, dilation_rate=4, activation='relu', name='Conv1/4_22')(y)
    y = BatchNormalization(name='BN_32')(y)
    y = Conv2D(1024, 1, name='Conv1/4_23')(y)
    y = BatchNormalization(name='BN_33')(y)
    y = Add(name='Add9')([y,y_])
    y_ = Activation('relu', name='RELU_9')(y)

    y = Conv2D(256, 1, activation='relu', name='Conv1/4_24')(y_)
    y = BatchNormalization(name='BN_34')(y)
    y = ZeroPadding2D(padding=4, name='Padding10')(y)
    y = Conv2D(256, 3, dilation_rate=4, activation='relu', name='Conv1/4_25')(y)
    y = BatchNormalization(name='BN_35')(y)
    y = Conv2D(1024, 1, name='Conv1/4_26')(y)
    y = BatchNormalization(name='BN_36')(y)
    y = Add(name='Add10')([y,y_])
    y_ = Activation('relu', name='RELU_10')(y)

    y = Conv2D(256, 1, activation='relu', name='Conv1/4_27')(y_)
    y = BatchNormalization(name='BN_37')(y)
    y = ZeroPadding2D(padding=4, name='Padding11')(y)
    y = Conv2D(256, 3, dilation_rate=4, activation='relu', name='Conv1/4_28')(y)
    y = BatchNormalization(name='BN_38')(y)
    y = Conv2D(1024, 1, name='Conv1/4_29')(y)
    y = BatchNormalization(name='BN_39')(y)
    y = Add(name='Add11')([y,y_])
    y = Activation('relu', name='RELU_11')(y)

    h, w = y.shape[1:3].as_list()
    pool1 = AveragePooling2D(pool_size=(h,w), strides=(h,w), name='Avg_Pool_1')(y)
    pool1 = Lambda(Resizer2, arguments={'value':[h,w]}, name='Resizer3')(pool1)
    pool2 = AveragePooling2D(pool_size=(h/2,w/2), strides=(h//2,w//2), name='Avg_Pool_2')(y)
    pool2 = Lambda(Resizer2, arguments={'value':[h,w]}, name='Resizer4')(pool2)
    pool3 = AveragePooling2D(pool_size=(h/3,w/3), strides=(h//3,w//3), name='Avg_Pool_3')(y)
    pool3 = Lambda(Resizer2, arguments={'value':[h,w]}, name='Resizer5')(pool3)
    pool6 = AveragePooling2D(pool_size=(h/4,w/4), strides=(h//4,w//4), name='Avg_Pool_4')(y)
    pool6 = Lambda(Resizer2, arguments={'value':[h,w]}, name='Resizer6')(pool6)

    y = Add(name='Add12')([y, pool1, pool2, pool3, pool6])
    y = Conv2D(256, 1, activation='relu', name='Conv1/4_30')(y)
    y = BatchNormalization(name='BN_40')(y)
    aux_1 = Lambda(Resizer3, name='Resizer7')(y)
    y = ZeroPadding2D(padding=2, name='Padding12')(aux_1)
    y = Conv2D(128, 3, dilation_rate=2, name='Conv1/4_31')(y)
    y = BatchNormalization(name='BN_41')(y)
    y_ = Conv2D(128, 1, name='Conv1/4_32')(z)
    y_ = BatchNormalization(name='BN_42')(y_)
    y = Add(name='Add13')([y,y_])
    y = Activation('relu', name='RELU_12')(y)

    aux_2 = Lambda(Resizer3, name='Resizer8')(y)
    y = ZeroPadding2D(padding=2, name='Padding13')(aux_2)
    y_ = Conv2D(128, 3, dilation_rate=2, name='Conv1/4_33')(y)
    y_ = BatchNormalization(name='BN_43')(y_)

    # (1)
    y = Conv2D(32, 3, strides=2, padding='same', activation='relu', name='Conv1_1')(x)
    y = BatchNormalization(name='BN_44')(y)
    y = Conv2D(32, 3, strides=2, padding='same', activation='relu', name='Conv1_2')(y)
    y = BatchNormalization(name='BN_45')(y)
    y = Conv2D(64, 3, strides=2, padding='same', activation='relu', name='Conv1_3')(y)
    y = BatchNormalization(name='BN_46')(y)
    y = Conv2D(128, 1, name='Conv1_4')(y)
    y = BatchNormalization(name='BN_47')(y)

    y = Add(name='sub12_sum')([y,y_])
    y = Activation('relu', name='RELU_13')(y)
    y = Lambda(Resizer3, name='Resizer9')(y)
    
    n_classes=3
    
    out = Conv2D(n_classes, 1, activation='softmax', name='Out3')(y)
    
    if train:
        aux_1 = Conv2D(n_classes, 1, activation='softmax', name='Out1')(aux_1)
        aux_2 = Conv2D(n_classes, 1, activation='softmax', name='Out2')(aux_2)
        model = Model(inputs=inp, outputs=[out, aux_2, aux_1])
        
    return model

### Custom cost function 

In [ ]:
from keras import backend as K
def weighted_binary_crossentropy():
    def loss(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        weights=tf.count_nonzero(y_true,axis=1,keep_dims=True)
        weights=tf.to_float(weights)
        loss = (y_true * K.log(y_pred)+(1-y_true)*K.log(1-y_pred))*(1-(weights/tf.to_float(tf.size(y_true)/3)))
        loss = -K.sum(loss, -1)
        return loss    
    return loss

### Define and fit the model 

In [ ]:
model=network(800,384,train=True)
model.compile(loss=weighted_binary_crossentropy(),loss_weights=[10,4.0,4.0],optimizer='adam',metrics=['accuracy'])
nb_data=200
tbCallBack=TensorBoard(log_dir='./Keras_Log3', histogram_freq=0, write_graph=True, write_images=True)
class_weight={0:1,1:2,2:2.5}
with tf.device("/gpu:0"):
    model.fit_generator(
            generate_arrays_from_file('Train/Train/Files31.txt','Train/Train/CameraRGB','Train/Train/CameraSeg' ),
            steps_per_epoch=nb_data,shuffle=True, validation_data=generate_arrays_from_file('Train/Train/Files31.txt','Train/Train/CameraRGB','Train/Train/CameraSeg' ), validation_steps=10,
            epochs=100,callbacks=[tbCallBack])

### Save model architecture and weights 

In [ ]:
model_yaml = model.to_yaml()
with open("model_architecture.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
model.save_weights("model_weights.h5")
print("Saved model to disk")